In [1]:
"""
This script is used for cleaning data in local fields

"""
import pandas as pd
import gspread as gs
import numpy as np
import matplotlib
import requests
import json
import datetime
import sys
import os

In [2]:
libraries_data = r'N:\Dossiers personnels\Mottet\python\python_scripts\script\project\raw_data_ressources\libraries_rbnj.json'
item_types_data = r'N:\Dossiers personnels\Mottet\python\python_scripts\script\project\raw_data_ressources\item_types_rbnj.json'
locations_data = r'N:\Dossiers personnels\Mottet\python\python_scripts\script\project\raw_data_ressources\locations_rbnj.json'
patron_types_data = r'N:\Dossiers personnels\Mottet\python\python_scripts\script\project\raw_data_ressources\patron_types_rbnj.json'
circ_policies_data = r'N:\Dossiers personnels\Mottet\python\python_scripts\script\project\raw_data_ressources\circ_policies_rbnj.json'
librarians_data = r'N:\Dossiers personnels\Mottet\python\python_scripts\script\project\raw_data_ressources\librarians_rbnj.json'

naming = [(libraries_data, "library.pid"),(item_types_data, "item_type.pid"), (item_types_data, "temporary_item_type.pid"),(locations_data, "location.pid"), (locations_data, "temporary_location.pid")]

"""def jsonApiToDataframe(search):
    with open(search, 'r', encoding='utf-8') as file:
        data = json.load(file)
        
        print("json load ok")
    try:
        df = pd.json_normalize(data["hits"]["hits"], max_level=10, errors='ignore')
        print("json normalize ok")
        print(f"taille = {df.shape}")
    except:
        df = pd.json_normalize(data, max_level=10, errors='ignore')
        print("json normalize ok")
        print(f"taille = {df.shape}")
    return df"""

def jsonApiToDataframe(search):
    dataframes = {}
    basename = os.path.basename(search)
    if ".json" in search:
        with open(search, 'r', encoding='utf-8') as file:
            data = json.load(file)
            print(f"{basename} json load ok")
        try:
            df = pd.json_normalize(data["hits"]["hits"], max_level=10, errors='ignore')
            print(f"{basename} json normalize ok")
            print(f"{basename} taille = {df.shape}")
            dataframes.update({basename: df})
        except:
            df = pd.json_normalize(data, max_level=10, errors='ignore')
            print(f"{basename} json normalize ok")
            print(f"{basename} taille = {df.shape}")
            dataframes.update({basename: df})
    else:
        for filename in os.listdir(search):
            f = os.path.join(search, filename)
            with open(f, 'r', encoding='utf-8') as file:
                data = json.load(file)
                print(f"{basename} json load ok")
            try:                
                df = pd.json_normalize(data['hits']['hits'], max_level=10, errors='ignore')
                print(f"{basename} json normalize ok")
                print(f"{basename} taille = {df.shape}")
                dataframes.update({filename: df})
            except:
                df = pd.json_normalize(data, max_level=10, errors='ignore')
                print(f"{basename} json normalize ok")
                print(f"{basename} taille = {df.shape}")
                dataframes.update({filename: df})
    print(df.shape)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    display(df.tail())
    return dataframes

def pidRenaming(df):
    """- keep only pid in column and rename column with the pattern ..._pid
       - delete "organisation.$ref" column
    
    """
    if 'organisation.$ref' in df.columns.values:
        df.drop('organisation.$ref', axis=1, inplace=True)
    for column in df.columns.values:
        if "$ref" in column:
            df['ressource_type'] = df[column].str.extract(r'^https://bib.rero.ch/api/([a-z]*)/', expand = True)
            df[column] = df[column].str.extract("(\d+)", expand=False)
            column_split = column.split(".")
            column_pid_rename = f"{''.join(column_split[-3:-2])}{''.join(column_split[-2:-1])}.pid"
            df.rename(columns = {column:column_pid_rename}, inplace = True)
    return df

def addColumn(origine_df, merged_df, nameMergedColumn, mergedColumnRenaming, origineJoinColumn):
    """add a column from another df"""
    if nameMergedColumn in origine_df:
        print(f"{nameMergedColumn} déjà dans {origine_df}")
    else:
        with open(merged_df, 'r', encoding='utf-8') as file:
            data = json.load(file)
        dfAddColumn = pd.json_normalize(data["hits"]["hits"], max_level=10, errors='ignore')
        dfAddColumn = dfAddColumn.rename(columns={'id': 'id_add'})
        dfAddColumn = dfAddColumn.filter(['id_add', nameMergedColumn])
        print(dfAddColumn.shape)
        print("dfAddColumn2")
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        display(dfAddColumn.tail())

        dfAddColumn = dfAddColumn.astype({'id_add': 'float64'}, errors='ignore')
        origine_df = origine_df.astype({origineJoinColumn: 'float64'}, errors='ignore')
        origine_df = origine_df.merge(dfAddColumn, how='left', left_on=origineJoinColumn, right_on='id_add')    
        origine_df = origine_df.drop(columns="id_add")
        origine_df.rename(columns = {nameMergedColumn: mergedColumnRenaming}, inplace = True)
        print(df.shape)
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        display(df.tail())
    return origine_df

def addNameColumn(naming, df):
    """add column with name based on pid"""    
    for items in naming:
        data_ressource, column_fusion = items
        if column_fusion in df.columns:
            dh = jsonApiToDataframe(data_ressource)
            
            dg = list(dh.values())
            dataframeName = list(dh.keys())
            
            column_name_rename = f"{column_fusion}_name"
            if column_name_rename not in df.columns.values.tolist():
                dg_name = dg[0].filter(['id', 'metadata.name'])
                dg_name = dg_name.rename(columns={'id': 'id_add'})
                dg_name = dg_name.astype({'id_add': 'float64'}, errors='ignore')

                df = df.astype({column_fusion: 'float64'}, errors='ignore')
                print(df.shape)
                print("df")
                pd.set_option('display.max_rows', None)
                pd.set_option('display.max_columns', None)
                display(df.tail())
                
                print(dg_name.shape)
                print("dg_name")
                pd.set_option('display.max_rows', None)
                pd.set_option('display.max_columns', None)
                display(dg_name.tail())
                
                df = df.merge(dg_name, how='left', left_on=column_fusion, right_on='id_add')
                df = df.drop(columns="id_add")

                df = df.rename(columns={"metadata.name": column_name_rename})
                print(f"colonne {column_name_rename} ajoutée")
            else:
                print(f"Colonne {column_name_rename} déjà présente")
        else:
            print(f"Pas de colonne {column_fusion} dans le dataframe")
    
    print(df.shape)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    display(df.tail())
    return df

def parsingBarecode (df, automaticBarcode : bool):
    """select all wrong barcode for an institution and save the list in Excel file"""

    ###A contrôler
    #bibJubicj #'^070\d{5}$'
    #bibJuaaeb #?
    #bibNeheaa #exclude range 1051226900 to 1051231899
    #bibNeacae #1040\d{6}|   |1049\d{6}|1054\d{6}
    #bibNebpun #a voir #|03\d{7} !!!! a contrôler
    #bibNebpup #a voir #|03\d{7} !!!! a contrôler
    #bibNecmh #?
    #bibJutpi '^080\d{5}$'

    automaticBarecode = '^RERO-|nej|f-\d{20}$'
    barcodeInstitutions = [("bibJuaaeb", 27, ''), 
                        ("bibJubicj", 29, '^010\d{5}|1048\d{6}$'), 
                        ("bibJubno", 60, ''), 
                        ("bibNeheaa", 47, '^1057\d{6}|1051\d{6}|105122690-90-9|1051227-90-90-90-9|10512300-90-90-9|10512310-80-90-9$'),  
                        ("bibNecfbv", 30, '^1050\d{6}|1051\d{6}|1056\d{6}|1059\d{6}|05\d{7}|10544\d{5}$'), 
                        ("bibNelocl", 31, '^1055\d{6}$'), 
                        ("bibNeacae", 28, '^1041\d{6}$'), 
                        ("bibNecfbj2", 33, '^1058\d{6}$'), 
                        ("bibNecfbj1", 32, '^1058\d{6}$'), 
                        ("bibJubmda", 36, '^700\d{5}|701\d{5}|0013770\d{7}|GL0\d{5}$'), 
                        ("bibJubmdj", 37, '^700\d{5}|701\d{5}|0013770\d{7}|GL0\d{5}$'), 
                        ("bibNebpun", 38, '^1030\d{6}|1031\d{6}|1032\d{6}|1039\d{6}|1041\d{6}|3200000\d{7}$'), 
                        ("bibNebpup", 39, '^1030\d{6}|1031\d{6}|1032\d{6}|1039\d{6}|3200000\d{7}$'), 
                        ("bibJusad", 55, '^070\d{5}$'), 
                        ("bibBemcip", 44, '^CIP\d{5}|0\d{5}|4\d{5}|5\d{5}|7\d{5}$'), 
                        ("bibJucold", 45, '^1047\d{6}$'), 
                        ("bibJucsch", 35, '^1047\d{6}$'), 
                        ("bibJuecd", 48, '^040\d{5}$'), 
                        ("bibJuecg", 46, '^030\d{5}$'), 
                        ("bibNecfesB", 40, '^10548\d{5}$'), 
                        ("bibNecfesN", 41, '^105490\d{4}$'), 
                        ("bibNecfesC", 42, '^105495\d{4}$'), 
                        ("bibNecfesF", 43, '^10547\d{5}$'), 
                        ("bibJufric", 49, '^050\d{5}$'), 
                        ("bibJulcp", 50, '^060\d{5}$'), 
                        ("bibNecmih", 52, '^10546\d{5}$'), 
                        ("bibNemahn", 51, '^1033\d{6}$'), 
                        ("bibNecmh", 57, ''), 
                        ("bibNecmhn", 58, '^10544\d{5}$'), 
                        ("bibNecmba", 56, '^10540\d{5}$'), 
                        ("bibJutc", 34, '^08\d{6}$'), 
                        ("bibJutpi", 54, '')]
    
   
    #dh = pd.DataFrame (barcodeInstitutions, columns = ['lib_code', 'lib_pid', 'patterns'])
    
    
    for items in barcodeInstitutions:
        dg = df.copy()
        lib_code, lib_pid, patterns = items
        dg = dg.loc[dg['library_pid'] == lib_pid]
        dg = (dg[~dg.barcode.str.contains(pat = patterns, regex = True, na = False)])
        
        print(dg.shape)
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', 300)
        display(dg.head(10))
        
    """if automaticBarcode == True:
        dg = df.copy()
        dg = (dg[~dg.barcode.str.contains(pat = automaticBarecode, regex = True, na = False)])"""
        


    """dg = [(dg[~dg.barcode.str.contains(pat = patterns, regex = True, na = False)]) for pattern in patterns]"""
    
    """
    for items in barcodeInstitutions:
        lib_code, lib_pid, patterns = items
        if automaticBarcode != True:
            #dg = [lib_pid for lib_pid in dg['library_pid']]
            dg = dg.loc[dg['library_pid'] == lib_pid]
        dg = [(dg[~dg.barcode.str.contains(pat = patterns, regex = True, na = False)]) for pattern in patterns]"""

          
    """dg = dg.filter(['pid', 'barcode', 'acquisition_date', 'call_number', 'library_pid'])
    dg.to_excel(fr"N:\\Dossiers personnels\\Mottet\\python\\python_scripts\\script\\project\\all_items_RBNJ\\{lib_code}_wrong_barcodes.xlsx")"""
    return dg

def saveToExcel(toSave, savePath):
    """savec all df to Excel"""
    if type(toSave) == dict:
        for filename, value in toSave.items():
            save = f"{savePath}\\{filename}.xlsx"
            print(f"file {filename}.xlsx save at {save}")
            value.to_excel(save)
    else:
        save = f"{savePath}\output.xlsx"
        toSave.to_excel(save)
        print(f"file save at {save}")
    return


In [3]:
#Import Gsheet as dataframe

gc = gs.service_account(filename='N:\Informatique_bureautique\gestion\code\python\gsheet_key.json')
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/18dNITSwMePVzH3-aEEZ4vA5Tcv5geybtXPxncIOMdoQ/edit#gid=1384104718')
ws = sh.worksheet('Champs_locaux')
local_fields_data = pd.DataFrame(ws.get_all_records())

#change some values in fields
transform_fields_name = [
("champ_1", "fields.field_1"),
("champ_2", "fields.field_2"),
("champ_3", "fields.field_3"),
("champ_4", "fields.field_4"),
("champ_5", "fields.field_5"),
("champ_6", "fields.field_6"),
("champ_7", "fields.field_7"),
("champ_8", "fields.field_8"),
("champ_9", "fields.field_9"),
("champ_10", "fields.field_10"),
("sans_valeur", "NaN"),
("zone_019", "zone_019")
]
for field_gsheet, field_df in transform_fields_name:
    local_fields_data['champs'] = local_fields_data['champs'].replace(field_gsheet, field_df)

#print(local_fields_data.shape)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_colwidth', 500)
#display(local_fields_data.head(20))


In [6]:
#items not migrated from Virtua
#search = r'N:\Dossiers personnels\Mottet\python\python_scripts\script\project\data_for_missed_item_from_Virtua_in RERO_ILS\input\rbnj_items_to_reload.json'
search = r'N:\Informatique_bureautique\gestion\code\johan\python_scripts\script\project\local_fields\input\local_fields_2022-10-19_corection.json'
merged_df = r'N:\Informatique_bureautique\gestion\code\johan\python_scripts\script\project\raw_data_ressources\locations_rbnj.json'
savepath = r"N:\Dossiers personnels\Mottet\python\python_scripts\script\project\data_for_missed_item_from_Virtua_in RERO_ILS"
nameMergedColumn = 'metadata.library.pid'
mergedColumnRenaming = "library_pid"
origineJoinColumn = "metadata.location.pid"

df = jsonApiToDataframe(search)
dg = list(df.values())
df = pidRenaming(dg[0])

dg = df.copy()
#print(dg.dtypes)

#print(dg.shape)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_colwidth', 500)
#display(dg.tail(10))

str_columns = ['fields.field_3', 'fields.field_1', 'fields.field_2']

dataframe_list = []
for column in str_columns:
    dg = dg.astype({column: 'str'})
    #delete [] at the end and begining of the field
    dg[column] = dg[column].str.slice(start=2, stop=-2)

    #separate content separte by | or , in deferente row. Select useful columns, Save each splited column in a dataframe
    dg[column] = dg[column].str.split("\', \'|\|", regex=True, expand=False)
    dg_fields = dg.filter(['pid', 'parent.pid', 'ressource_type', column])
    dg_fields = dg_fields.explode(column)
    #save all dataframes in a list
    dataframe_list.append(dg_fields)
    
#print(dataframe_list[2].shape)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_colwidth', 500)
#display(dataframe_list[2].tail(10))


#add the field "field_name", rename the field in "content", concatenate all the dataframes 
for dataframe in dataframe_list:
    series_list = dataframe.columns.values.tolist()
    dataframe.insert(4, "field_name", series_list[3])
    dataframe.rename(columns = {list(dataframe)[3]:'content'}, inplace = True)
    big_df = pd.concat(dataframe_list)
    
    
    #print(dataframe.shape)
    #pd.set_option('display.max_rows', None)
    #pd.set_option('display.max_columns', 500)
    #pd.set_option('display.max_colwidth', 500)
    #display(dataframe.tail(10))

#delete empty rows
df_raw = big_df[~big_df['content'].isin([''])]

#put columns in a good order
column_names = ["pid", "parent.pid", "ressource_type", "field_name", "content"]
df_raw = df_raw.reindex(columns=column_names)

#remove $a and $2 at field begining + space. Reset index. Add column remarks
df_raw['content'] = df_raw['content'].str.strip()
df_raw['content'] = df_raw['content'].str.replace(r'^\$a','', regex=True)
df_raw['content'] = df_raw['content'].str.replace(r'^\$2','', regex=True)
df_raw['content'] = df_raw['content'].str.strip()
df_raw = df_raw.reset_index(drop=True)
df_raw.insert(4, "remarks", "")


#print(df_raw.shape)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_colwidth', 500)
#display(df_raw.head(20))

##Create a copy
df_duplicate = df_raw.copy()

local_fields_2022-10-19_corection.json json load ok
local_fields_2022-10-19_corection.json json normalize ok
local_fields_2022-10-19_corection.json taille = (768982, 6)
(768982, 6)


,pid,fields.field_3,parent.$ref,organisation.$ref,fields.field_1,fields.field_2
768977,1460097,NaN,https://bib.rero.ch/api/documents/794839,https://bib.rero.ch/api/organisations/3,[$a juescdna $b 2007],[$2 cdu-juescd $a 616 POMM | $2 jubmd $a 616.8...
768978,1846228,[$a CFV 301.412 HER $x -],https://bib.rero.ch/api/documents/2132859,https://bib.rero.ch/api/organisations/3,[$a necfbvna $b 2212 $c 1341],NaN
768979,1850126,[$a CFV 301.153 WUM $x -],https://bib.rero.ch/api/documents/2137031,https://bib.rero.ch/api/organisations/3,[$a necfbvna $b 2212 $c 123],NaN
768980,1842325,[$a CFV NOTH L $x - | $a BPUP R NOTH Liv $x ne...,https://bib.rero.ch/api/documents/2128546,https://bib.rero.ch/api/organisations/3,[$a necfbvna $b 2209 $c 119 | $a bpunlp $b 221...,NaN
768981,1846221,[$a CFV 613.88 RAM $x - | $a BPUP 613.885 RAMA...,https://bib.rero.ch/api/documents/2132851,https://bib.rero.ch/api/organisations/3,[$a necfbvna $b 2212 $c 1341 | $a bpunlp $b 22...,NaN


(768982, 6)


,pid,fields.field_3,parent.pid,fields.field_1,fields.field_2,ressource_type
768977,1460097,NaN,794839,[$a juescdna $b 2007],[$2 cdu-juescd $a 616 POMM | $2 jubmd $a 616.8...,documents
768978,1846228,[$a CFV 301.412 HER $x -],2132859,[$a necfbvna $b 2212 $c 1341],NaN,documents
768979,1850126,[$a CFV 301.153 WUM $x -],2137031,[$a necfbvna $b 2212 $c 123],NaN,documents
768980,1842325,[$a CFV NOTH L $x - | $a BPUP R NOTH Liv $x ne...,2128546,[$a necfbvna $b 2209 $c 119 | $a bpunlp $b 221...,NaN,documents
768981,1846221,[$a CFV 613.88 RAM $x - | $a BPUP 613.885 RAMA...,2132851,[$a necfbvna $b 2212 $c 1341 | $a bpunlp $b 22...,NaN,documents


In [7]:
#indicate content in the wrong field in remarks field -> peut-être à utiliser uniquement après exclusion des champs mal formés

##filter gsheet dataframe to only have fields.field_X and extract in a list of tuple regex and field
bad_field_content = local_fields_data[~(local_fields_data['champs'].str.contains(pat = "NaN|zone_019", regex = True))]
regex_field = list(zip(bad_field_content['regex'],bad_field_content['champs']))

#detect content that fit regex but not the field
for regex, field in regex_field:
    transfer = df_duplicate[(~(df_duplicate['field_name']==field)&(df_duplicate['content'].str.contains(pat = regex, regex = True, na = False)))]   # Add document column |~((df_duplicate['ressource_type']=='documents')&(df_duplicate['content'].str.contains(pat = regex, regex = True, na = False))
    index_transfer = transfer.index.values.tolist()
    
    #Add right field in remarks field
    for no_index in index_transfer:
        df_duplicate.at[no_index, 'remarks'] = fr"Transferer le champs dans la ressource 'Document', {field}"
        #print(fr"Le contenu '{transfer.loc[no_index, 'content']}' dans le champ '{transfer.loc[no_index, 'field_name']}', ressource '{transfer.loc[no_index, 'ressource_type']}' doit aller dans le champ '{field}' de la ressource 'documents'.")

for regex, field in regex_field:
    transfer = df_raw[(~(df_raw['field_name']==field)&(df_raw['content'].str.contains(pat = regex, regex = True, na = False)))]   # Add document column |~((df_duplicate['ressource_type']=='documents')&(df_duplicate['content'].str.contains(pat = regex, regex = True, na = False))
    index_transfer = transfer.index.values.tolist()
    
    #Add right field in remarks field
    for no_index in index_transfer:
        df_raw.at[no_index, 'remarks'] = fr"Transferer le champs dans la ressource 'Document', {field}"
        #print(fr"Le contenu '{transfer.loc[no_index, 'content']}' dans le champ '{transfer.loc[no_index, 'field_name']}', ressource '{transfer.loc[no_index, 'ressource_type']}' doit aller dans le champ '{field}' de la ressource 'documents'.")
 

C:\Users\mottetj\AppData\Local\Temp\ipykernel_1564\3791976585.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  transfer = df_duplicate[(~(df_duplicate['field_name']==field)&(df_duplicate['content'].str.contains(pat = regex, regex = True, na = False)))]   # Add document column |~((df_duplicate['ressource_type']=='documents')&(df_duplicate['content'].str.contains(pat = regex, regex = True, na = False))
C:\Users\mottetj\AppData\Local\Temp\ipykernel_1564\3791976585.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  transfer = df_raw[(~(df_raw['field_name']==field)&(df_raw['content'].str.contains(pat = regex, regex = True, na = False)))]   # Add document column |~((df_duplicate['ressource_type']=='documents')&(df_duplicate['content'].str.contains(pat = regex, regex = True, na = False))


In [8]:
dupl = df_duplicate[df_duplicate['remarks'].str.contains('Transferer')]
print(dupl.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
display(dupl.head(20))

(1785343, 6)


,pid,parent.pid,ressource_type,field_name,remarks,content
0,1171616,253332,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",CMHN Combles est $x --
1,1170497,251516,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP PT 421/7009 $x ne/bpun/a [Sans les diapositives.]
2,1170429,251397,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP PT 744/386 $x ne/bpun/a/1992/1017
3,1170429,251397,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP FP 2132 $x ne/bpun/a/1992/1018
4,1170429,251397,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NELA 32419 $x ne/ubfl/a
5,1170752,251952,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",842/698 $x ne/bpup/a/1992
6,1224574,342308,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",BPUN QU 718/13 $x ne/bpun/d
7,1224574,342308,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",PU 3088/13 $x ne/bpun/d
8,1224574,342308,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NET 701/13 $x ne/ueie/e
9,1171383,252964,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP PT 581/8 $x ne/bpun/a/1992/1629


In [9]:
#Signal and delete abandonned content

##select only content abandoned
no_rero_ils_content = local_fields_data.loc[local_fields_data['statut']=='Code abandonné']

##select local fields dataframe with abandoned regex
abandoned_rows = (df_duplicate[df_duplicate.content.str.contains('|'.join(no_rero_ils_content['regex']), regex = True, na = False)])
index_abandoned_rows = abandoned_rows.index.values.tolist()

## Add "Contenu abandonné" in field "Remarks" in df_raw et df_duplicate Dataframes
for no_index in index_abandoned_rows:
    df_raw.at[no_index, 'remarks'] = "Contenu abandonné"
    df_duplicate.at[no_index, 'remarks'] = "Contenu abandonné"

## Delete abandonned fields from df_duplicate dataframe
df_duplicate = (df_duplicate[~df_duplicate.remarks.str.contains("Contenu abandonné")])

dupl = df_duplicate[df_duplicate['remarks'].str.contains('Transferer')]
print(dupl.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
display(dupl.head(20))

(1618937, 6)


,pid,parent.pid,ressource_type,field_name,remarks,content
0,1171616,253332,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",CMHN Combles est $x --
1,1170497,251516,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP PT 421/7009 $x ne/bpun/a [Sans les diapositives.]
2,1170429,251397,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP PT 744/386 $x ne/bpun/a/1992/1017
3,1170429,251397,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP FP 2132 $x ne/bpun/a/1992/1018
5,1170752,251952,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",842/698 $x ne/bpup/a/1992
6,1224574,342308,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",BPUN QU 718/13 $x ne/bpun/d
7,1224574,342308,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",PU 3088/13 $x ne/bpun/d
9,1171383,252964,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP PT 581/8 $x ne/bpun/a/1992/1629
11,1171383,252964,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",PA: SR 147/8.1 $x ne/past/a/39481
12,1170635,251760,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP QU 132/31 $x ne/bpun/d/n


In [10]:
#content with hors-rero institutions code
potentialy_to_remove = [
'^(?!CFV)[\s\S]*gpsr[\s\S]*$',
'^(?!CFV)[\s\S]*hepbjnfusion[\s\S]*$',
'^(?!CFV)[\s\S]*irdp[\s\S]*$',
'^(?!CFV)[\s\S]*juarcd[\s\S]*$',
'^(?!CFV)[\s\S]*juhepma[\s\S]*$',
'^(?!CFV)[\s\S]*neacsj[\s\S]*$',
'^(?!CFV)[\s\S]*neacsjna[\s\S]*$',
'^(?!CFV)[\s\S]*nearcn[\s\S]*$',
'^(?!CFV)[\s\S]*nearcnna[\s\S]*$',
'^(?!CFV)[\s\S]*neccm[\s\S]*$',
'^(?!CFV)[\s\S]*neccmna[\s\S]*$',
'^(?!CFV)[\s\S]*neciesna[\s\S]*$',
'^(?!CFV)[\s\S]*negpsrna[\s\S]*$',
'^(?!CFV)[\s\S]*nehecr[\s\S]*$',
'^(?!CFV)[\s\S]*nehecrna[\s\S]*$',
'^(?!CFV)[\s\S]*nehegn[\s\S]*$',
'^(?!CFV)[\s\S]*nehegnna[\s\S]*$',
'^(?!CFV)[\s\S]*neirdpna[\s\S]*$',
'^(?!CFV)[\s\S]*nemhnnna[\s\S]*$',
'^(?!CFV)[\s\S]*nencm[\s\S]*$',
'^(?!CFV)[\s\S]*nencmna[\s\S]*$',
'^(?!CFV)[\s\S]*netcne[\s\S]*$',
'^(?!CFV)[\s\S]*netcnena[\s\S]*$',
'^(?!CFV)[\s\S]*neu-sfm[\s\S]*$',
'^(?!CFV)[\s\S]*neubfd[\s\S]*$',
'^(?!CFV)[\s\S]*neubfdna[\s\S]*$',
'^(?!CFV)[\s\S]*neubfl[\s\S]*$',
'^(?!CFV)[\s\S]*neubflbi[\s\S]*$',
'^(?!CFV)[\s\S]*neubflborie[\s\S]*$',
'^(?!CFV)[\s\S]*neubfleaha[\s\S]*$',
'^(?!CFV)[\s\S]*neubfleahiam[\s\S]*$',
'^(?!CFV)[\s\S]*neubfleahimc[\s\S]*$',
'^(?!CFV)[\s\S]*neubflealcal[\s\S]*$',
'^(?!CFV)[\s\S]*neubflealcfr[\s\S]*$',
'^(?!CFV)[\s\S]*neubflealo[\s\S]*$',
'^(?!CFV)[\s\S]*neubfleashsge[\s\S]*$',
'^(?!CFV)[\s\S]*neubfleashsmi[\s\S]*$',
'^(?!CFV)[\s\S]*neubfleashsso[\s\S]*$',
'^(?!CFV)[\s\S]*neubfleaslclc[\s\S]*$',
'^(?!CFV)[\s\S]*neubflei[\s\S]*$',
'^(?!CFV)[\s\S]*neubfleigeldinger[\s\S]*$',
'^(?!CFV)[\s\S]*neubflfasani[\s\S]*$',
'^(?!CFV)[\s\S]*neubflgagnebin[\s\S]*$',
'^(?!CFV)[\s\S]*neubflna[\s\S]*$',
'^(?!CFV)[\s\S]*neubflrecht[\s\S]*$',
'^(?!CFV)[\s\S]*neubflrück[\s\S]*$',
'^(?!CFV)[\s\S]*neubflschüle[\s\S]*$',
'^(?!CFV)[\s\S]*neubflseyrig[\s\S]*$',
'^(?!CFV)[\s\S]*neubflstuder[\s\S]*$',
'^(?!CFV)[\s\S]*neubfluhler[\s\S]*$',
'^(?!CFV)[\s\S]*neubfsna[\s\S]*$',
'^(?!CFV)[\s\S]*neubfuhler[\s\S]*$',
'^(?!CFV)[\s\S]*neublfstuder[\s\S]*$',
'^(?!CFV)[\s\S]*neuetg[\s\S]*$',
'^(?!CFV)[\s\S]*neuethna[\s\S]*$',
'^(?!CFV)[\s\S]*neuidsna[\s\S]*$',
'^(?!CFV)[\s\S]*neulapna[\s\S]*$',
'^(?!CFV)[\s\S]*neulcdna[\s\S]*$',
'^(?!CFV)[\s\S]*neullcna[\s\S]*$',
'^(?!CFV)[\s\S]*neusbo[\s\S]*$',
'^(?!CFV)[\s\S]*neusesna[\s\S]*$',
'^(?!CFV)[\s\S]*neusfm[\s\S]*$',
'^(?!CFV)[\s\S]*neusfmna[\s\S]*$',
'^(?!CFV)[\s\S]*neusma[\s\S]*$',
'^(?!CFV)[\s\S]*neuthena[\s\S]*$',
'^(?!CFV)[\s\S]*njhep-migration[\s\S]*$',
'^(?!CFV)[\s\S]*njhepc[\s\S]*$',
'^(?!CFV)[\s\S]*njhepg[\s\S]*$',
'^(?!CFV)[\s\S]*njhepm[\s\S]*$',
'^(?!CFV)[\s\S]*njhepna[\s\S]*$',
'^(?!CFV)[\s\S]*subo[\s\S]*$',
'^(?!CFV)[\s\S]*sys-cdm[\s\S]*$',
'^(?!CFV)[\s\S]*ubfd[\s\S]*$',
'^(?!CFV)[\s\S]*ubfl[\s\S]*$',
'^(?!CFV)[\s\S]*ubo[\s\S]*$',
'^(?!CFV)[\s\S]*ueie[\s\S]*$',
'^(?!CFV)[\s\S]*ueme[\s\S]*$',
'^(?!CFV)[\s\S]*uemu[\s\S]*$',
'^(?!CFV)[\s\S]*us--[\s\S]*$',
'^(?!CFV)[\s\S]*usba[\s\S]*$',
'^(?!CFV)[\s\S]*usbi[\s\S]*$',
'^(?!CFV)[\s\S]*usbo[\s\S]*$',
'^(?!CFV)[\s\S]*usch[\s\S]*$',
'^(?!CFV)[\s\S]*uses[\s\S]*$',
'^(?!CFV)[\s\S]*usews[\s\S]*$',
'^(?!CFV)[\s\S]*usge[\s\S]*$',
'^(?!CFV)[\s\S]*ushy[\s\S]*$',
'^(?!CFV)[\s\S]*usma[\s\S]*$',
'^(?!CFV)[\s\S]*usme[\s\S]*$',
'^(?!CFV)[\s\S]*usmi[\s\S]*$',
'^(?!CFV)[\s\S]*usph[\s\S]*$',
'^(?!CFV)[\s\S]*uszo[\s\S]*$',
'^(?!CFV)[\s\S]*uthe[\s\S]*$',
'^(?!CFV)[\s\S]*uuses[\s\S]*$',
'^(?!CFV)[\s\S]*uzo[\s\S]*$',
'jubmd',
'jubmdare',
'jubmdjre',
'nepastna',
'jubmdana',
'jubmdana',
'jubmdjna',
'jubmdjna',
'jubmdjna'
]

##select local fields dataframe with abandoned regex
old_institutions_row = (df_duplicate[df_duplicate.content.str.contains('|'.join(potentialy_to_remove), na = False)])
index_old_institutions_rows = old_institutions_row.index.values.tolist()

print(old_institutions_row.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
display(old_institutions_row.head(20))

## Add "Contenu abandonné" in field "Remarks" in df_raw et df_duplicate Dataframes
for no_index in index_old_institutions_rows:
    df_raw.at[no_index, 'remarks'] = "Contenu abandonné"
    df_duplicate.at[no_index, 'remarks'] = "Contenu abandonné"

dupl = df_duplicate[df_duplicate['remarks'].str.contains('Transferer')]
print(dupl.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
display(dupl.head(20))

(33515, 6)


,pid,parent.pid,ressource_type,field_name,remarks,content
1633,1240188,369069,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",A-5-4-227 $x ne/neusge/a/11475
1749,1233773,358399,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",QL563 39.h $x neuszo
1769,1240554,369663,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",A-4-1-8 $x ne/usge/w/-
1848,1240534,369630,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",A-4-1-4 $x ne/usge/w/-
1852,1240877,370310,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",A-4-4-83 $x ne/usge/a/1713
2061,1238439,366022,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",B-1-1-7 $x ne/usge/d/11686
2275,1234684,359790,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",C-2-7-153 $x ne/usge/d/11658
2541,1242302,372615,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",A-8-9-2 $x ne/usge/a/7167
2560,1245770,378051,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",YS 35/2/64 ne/uses/d/53527 $x --
3024,1245359,377408,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",Ek 275 $x ne/usmi


(1585422, 6)


,pid,parent.pid,ressource_type,field_name,remarks,content
0,1171616,253332,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",CMHN Combles est $x --
1,1170497,251516,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP PT 421/7009 $x ne/bpun/a [Sans les diapositives.]
2,1170429,251397,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP PT 744/386 $x ne/bpun/a/1992/1017
3,1170429,251397,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP FP 2132 $x ne/bpun/a/1992/1018
5,1170752,251952,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",842/698 $x ne/bpup/a/1992
6,1224574,342308,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",BPUN QU 718/13 $x ne/bpun/d
7,1224574,342308,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",PU 3088/13 $x ne/bpun/d
9,1171383,252964,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP PT 581/8 $x ne/bpun/a/1992/1629
11,1171383,252964,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",PA: SR 147/8.1 $x ne/past/a/39481
12,1170635,251760,documents,fields.field_3,"Transferer le champs dans la ressource 'Document',",NP QU 132/31 $x ne/bpun/d/n


In [11]:


#Delete content well formed with regex. So We keep only content with mistakes.

##Select only row with regex in gsheet dataframe
wrong_writed_field_content = local_fields_data.loc[local_fields_data['regex']!='']                            
wrong_writed_field_regex = wrong_writed_field_content.regex.values.tolist()

#Delete content well formed with regex and in goog field
df_duplicate = df_duplicate[~(df_duplicate.content.str.contains(pat = '|'.join(wrong_writed_field_regex), regex = True, na = False))&
                             ~(df_duplicate.remarks.str.contains("Transferer le champs dans la ressource"))]

#doit être pris sur la même ligne pas sur la m'eme colonne
df_all_mistakes = df_raw.copy()
df_all_mistakes = df_all_mistakes[(df_all_mistakes.remarks.str.contains("Transferer"))|
                                  ~(df_all_mistakes.content.str.contains(pat = '|'.join(wrong_writed_field_regex), regex = True, na = False))]


#print(df_duplicate.shape)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_colwidth', 500)
#display(df_duplicate.head(20))




C:\Users\mottetj\AppData\Local\Temp\ipykernel_16848\110170903.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_duplicate = df_duplicate[~(df_duplicate.content.str.contains(pat = '|'.join(wrong_writed_field_regex), regex = True, na = False))&
C:\Users\mottetj\AppData\Local\Temp\ipykernel_16848\110170903.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ~(df_all_mistakes.content.str.contains(pat = '|'.join(wrong_writed_field_regex), regex = True, na = False))]


In [12]:
#signal deleted content in RERO ILS by adding "Contenu supprimé dans RERO ILS" in all dataframes df_raw and df_duplicate. Delete these content from df_duplicate
delete_row = [
#922206,
#209202,
#611011,
#133550,
#995343,
#165298,
#87845,
#154942,
#421556,
#783543,
#62105,
#2095084,
#483616,
#1389194,
#1522485
]

for no_index in delete_row:
    df_raw.at[no_index, 'remarks'] = "Contenu supprimé dans RERO ILS"
    df_duplicate.at[no_index, 'remarks'] = "Contenu supprimé dans RERO ILS"
    df_duplicate = (df_duplicate[~df_duplicate.remarks.str.contains("Contenu supprimé dans RERO ILS")])

#print(df_duplicate.shape)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_colwidth', 500)
#display(df_duplicate.head(20))

In [13]:
#add column occurences with numbre of occurences
df_duplicate['occurences'] = df_duplicate.groupby(['content'])['content'].transform('count')
df_raw['occurences'] = df_raw.groupby(['content'])['content'].transform('count')
df_all_mistakes['occurences'] = df_all_mistakes.groupby(['content'])['content'].transform('count')

##put column in good order
column_names = ["pid", "parent.pid", "ressource_type", "field_name", "occurences", "remarks", "content"]
df_duplicate = df_duplicate.reindex(columns=column_names)
df_raw = df_raw.reindex(columns=column_names)
df_all_mistakes = df_all_mistakes.reindex(columns=column_names)

df_all_mistakes.to_excel(r"N:\Dossiers personnels\Mottet\python\python_scripts\script\project\local_fields\output\df_all_mistakes.xlsx")

print(df_all_mistakes.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
display(df_all_mistakes.head(20))

ValueError: This sheet is too large! Your sheet size is: 1585913, 7 Max sheet size is: 1048576, 16384

In [ ]:
print(df_all_mistakes.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
display(df_all_mistakes.head(400))

In [ ]:
#export to excel df_raw. It's separated in 3 fields because dataframe is to big for Excel.
df_raw.sort_values(by=['content'])

filter_df_raw_1 = df_raw.loc[(df_raw['ressource_type']=='documents')&(df_raw['field_name']=='fields.field_1')]
filter_df_raw_2 = df_raw.loc[(df_raw['ressource_type']=='documents')&(df_raw['field_name']=='fields.field_2')]
filter_df_raw_3 = df_raw.loc[(df_raw['ressource_type']=='documents')&(df_raw['field_name']=='fields.field_3')]

filter_df_raw_1.to_excel("N:\Dossiers personnels\Mottet\python\python_scripts\script\project\local_fields\output\df_raw_1.xlsx")
filter_df_raw_2.to_excel("N:\Dossiers personnels\Mottet\python\python_scripts\script\project\local_fields\output\df_raw_2.xlsx")
filter_df_raw_3.to_excel("N:\Dossiers personnels\Mottet\python\python_scripts\script\project\local_fields\output\df_raw_3.xlsx")

In [ ]:
#delete duplicate content
df_duplicate = df_duplicate.drop_duplicates(subset=['content'])

print(df_duplicate.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
display(df_duplicate.head(20))

##save results
df_duplicate.to_excel(r"N:\Dossiers personnels\Mottet\python\python_scripts\script\project\local_fields\output\df_duplicate_no_duplicate.xlsx")

In [15]:
#add columns for field_3 
df_duplicate_field3 = df_duplicate.copy()

##extract first element from content for field 3 and add number of occurencies
df_duplicate_field3['start_field_3'] = df_duplicate_field3['content'].str.extract(r'(?:^)(\w+)[^\r\n]+')
df_duplicate_field3['start_field_3_occurences'] = df_duplicate_field3.groupby(['start_field_3'])['start_field_3'].transform('count')

##extract institution from content for field 3 and add number of occurencies
df_duplicate_field3['field_3_belonging'] = df_duplicate_field3['content'].str.extract(r'\$x ne\/([a-z\-]+)\/')
df_duplicate_field3['field_3_belonging_occurences'] = df_duplicate_field3.groupby(['field_3_belonging'])['field_3_belonging'].transform('count')

print(df_duplicate_field3.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
display(df_duplicate_field3.head(20))

##deduplicate start_field_3	
df_duplicate_field3 = df_duplicate_field3.sort_values(by='start_field_3_occurences', ascending=False)
df_duplicate_field3 = df_duplicate_field3.drop_duplicates(subset=['start_field_3'])

print(df_duplicate_field3.shape)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
display(df_duplicate_field3.head(20))

##save results
df_duplicate_field3.to_excel(r"N:\Dossiers personnels\Mottet\python\python_scripts\script\project\local_fields\output\field_3_start.xlsx")

(368, 11)


,pid,parent.pid,ressource_type,field_name,occurences,remarks,content,start_field_3,start_field_3_occurences,field_3_belonging,field_3_belonging_occurences
1633,1240188,369069,documents,fields.field_3,1,Contenu abandonné,A-5-4-227 $x ne/neusge/a/11475,A,65,neusge,1.0
1749,1233773,358399,documents,fields.field_3,1,Contenu abandonné,QL563 39.h $x neuszo,QL563,2,NaN,NaN
1769,1240554,369663,documents,fields.field_3,1,Contenu abandonné,A-4-1-8 $x ne/usge/w/-,A,65,usge,69.0
1848,1240534,369630,documents,fields.field_3,1,Contenu abandonné,A-4-1-4 $x ne/usge/w/-,A,65,usge,69.0
1852,1240877,370310,documents,fields.field_3,1,Contenu abandonné,A-4-4-83 $x ne/usge/a/1713,A,65,usge,69.0
2061,1238439,366022,documents,fields.field_3,1,Contenu abandonné,B-1-1-7 $x ne/usge/d/11686,B,1,usge,69.0
2275,1234684,359790,documents,fields.field_3,1,Contenu abandonné,C-2-7-153 $x ne/usge/d/11658,C,4,usge,69.0
2541,1242302,372615,documents,fields.field_3,1,Contenu abandonné,A-8-9-2 $x ne/usge/a/7167,A,65,usge,69.0
2560,1245770,378051,documents,fields.field_3,1,Contenu abandonné,YS 35/2/64 ne/uses/d/53527 $x --,YS,2,NaN,NaN
3024,1245359,377408,documents,fields.field_3,1,Contenu abandonné,Ek 275 $x ne/usmi,Ek,2,NaN,NaN


(94, 11)


,pid,parent.pid,ressource_type,field_name,occurences,remarks,content,start_field_3,start_field_3_occurences,field_3_belonging,field_3_belonging_occurences
908576,1027228,17052,documents,fields.field_1,2,Contenu abandonné,neusesna $b 2014/05 $c mj,neusesna,109,NaN,NaN
714883,1030118,20962,documents,fields.field_3,1,Contenu abandonné,A-9-6-32 $x ne/usge/a/4542,A,65,usge,69.0
946575,1698944,1622408,documents,fields.field_1,2,Contenu abandonné,jubmdana $b 1706,jubmdana,12,NaN,NaN
1055967,1566937,1211622,documents,fields.field_1,1,Contenu abandonné,njhepna $b 1309,njhepna,10,NaN,NaN
998976,1375663,587169,documents,fields.field_1,4,Contenu abandonné,juarcdna $b 2010/04,juarcdna,10,NaN,NaN
1058836,1727503,1696872,documents,fields.field_1,3,Contenu abandonné,neubflna $b 2019/1 $c ih,neubflna,9,NaN,NaN
906638,1460634,796142,documents,fields.field_1,1,Contenu abandonné,neuethna $b 2010/02,neuethna,7,NaN,NaN
1372755,1739917,1727325,documents,fields.field_1,1,Contenu abandonné,jubmdjna $b 19126 $c 01,jubmdjna,7,NaN,NaN
445118,1240433,369462,documents,fields.field_3,1,Contenu abandonné,I2.0/081 $x ne/usma,I2,5,NaN,NaN
1176406,1256950,398874,documents,fields.field_1,1,Contenu abandonné,neubfdna $b 1999/02 $c 05,neubfdna,5,NaN,NaN


In [ ]:
# Search bad fields separations. Detection of $a and $2 in content field can indicate a bad field separation

##remove known pattern with regex from gsheet
dollar_a_inside = local_fields_data[local_fields_data['regex'].str.contains('\$a|\$2', regex = True)]
dollara = (df_duplicate[~df_duplicate.content.str.contains(pat = '|'.join(dollar_a_inside['regex']), regex = True, na = False)])

##look for $a or $2 in the field
dollara = dollara[dollara['content'].str.contains('\$a|\$2', regex = True)]
#print(dollara.shape)
#display(dollara.head(20))

##save result
dollara.to_excel(r"N:\Dossiers personnels\Mottet\python\python_scripts\script\project\local_fields\output\bad_fields_separations.xlsx")


In [ ]:
#df with document from other institutions than RERO ILS

##Select only row with "hors RERO ILS" in gsheet dataframe
out_rero_ils_content = local_fields_data.loc[local_fields_data['reseau']=='hors RERO ILS']                            
out_rero_ils_regex = list[out_rero_ils_content['regex']]

##Select only regex with "hors RERO ILS"
only_out_rero_ils = (df_duplicate[df_duplicate.content.str.contains(pat = '|'.join(out_rero_ils_regex), regex = True, na = False)])

#print(only_SLSP.shape)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_colwidth', 500)
#display(only_SLSP.head(20))

##save results
#only_SLSP.to_excel("N:\Dossiers personnels\Mottet\python\python_scripts\script\project\local_fields\only_out_rero_ils.xlsx") 

In [ ]:
#df_raw.loc[df_raw['parent.pid']=='2069942']

remark_delete = "A été supprimé dans RERO ILS"
remark_transfer_to_document_field_1 = "A transférer dans la ressource 'document' champs 1"
remark_transfer_to_document_field_2 = "A transférer dans la ressource 'document' champs 2"
remark_transfer_to_document_field_3 = "A transférer dans la ressource 'document' champs 3"


#items, holdings, champ 3 sans $x


réguler les [\s\S] si erreur de "|"

Directives pour bibliothèques
    #marche à suivre correction manuel:
       # 2. controler que le bon format est respecté
       # 3. si nécessaire ajouter une ligne dans le tableau des champs
       # 4. contrôler si la séparation entre champ est correct "|"
       # 5. contrôler que la migration depuis Virtua s'est faite correctement

    le signe | sert de séparateur de champ. Il ne doit en aucun cas être utiliser pour autre chose.
    Ne rien mettre dans les champs locaux des holdings et des items

Demandes à RERO
    demander à RERO d'enlever tous les dollar a au début du champs et tus les espaces

    
1.identifier les valeurs uniques

2.détecter toutes les erreurs
 1.contenu mal formés
	-> mauvaise séparation "|" -> voir les contenu encore avec un $a
	-> erreurs de migration depuis Virtua
	-> erreurs de frappes
	
 2.contenu dans le mauvais champ
	-> a été automatisé avec les regex

    
    
comment fusionner le contenu de la 019 avec celui des champs locaux ?
Que deviennent les dollar de MARC ?
quelle syntaxe dans les champs ? un code doit toujours contenir la dénomionation de son institution
